In [6]:
import pandas as pd

geog = pd.read_csv('derived_data/geography.csv')

In [7]:
#https://hazards.fema.gov/nri/expected-annual-loss

df_nri_county = pd.read_csv("raw_data/fema_eal/NRI_Table_Counties/NRI_Table_Counties.csv")

nri_data_county = df_nri_county[[
    "STATEFIPS", "COUNTY", "STCOFIPS",
    "ALR_VALB", "ALR_VALP", "ALR_VALA"
]]

nri_data_county = nri_data_county.rename(columns={"STATEFIPS": "STATE", "COUNTY": "COUNTYNAME", "STCOFIPS": "COUNTY"})
nri_data_county["COUNTYNAME"] = nri_data_county["COUNTYNAME"].str.lower()

In [8]:
df_nri_tract = pd.read_csv("raw_data/fema_eal/NRI_Table_CensusTracts/NRI_Table_CensusTracts.csv")

nri_data_tract = df_nri_tract[[
    "TRACTFIPS",
    "ALR_VALB", "ALR_VALP", "ALR_VALA"
]]

nri_data_tract = nri_data_tract.rename(columns={"TRACTFIPS": "TRACT"})

In [9]:
nri_data_zip = df_nri_tract[[
    "TRACTFIPS",
    "ALR_VALB", "ALR_VALP", "ALR_VALA",
    "EAL_VALB", "EAL_VALP", "EAL_VALA",
    "POPULATION", "BUILDVALUE", "AGRIVALUE"
]]

nri_data_zip = nri_data_zip.rename(columns={"TRACTFIPS": "TRACT"})

nri_data_zip = nri_data_zip.merge(geog, on='TRACT')

nri_data_zip["EAL_VALB"] *= nri_data_zip["TZ_RATIO"]
nri_data_zip["EAL_VALP"] *= nri_data_zip["TZ_RATIO"]
nri_data_zip["EAL_VALA"] *= nri_data_zip["TZ_RATIO"]
nri_data_zip["POPULATION"] *= nri_data_zip["TZ_RATIO"]
nri_data_zip["BUILDVALUE"] *= nri_data_zip["TZ_RATIO"]
nri_data_zip["AGRIVALUE"] *= nri_data_zip["TZ_RATIO"]

gb = nri_data_zip.groupby("ZIP")
data = []
for zid, sub_df in gb:
    num, den = sub_df[["EAL_VALB", "BUILDVALUE"]].dropna().sum(axis=0).values
    ALR_VALB = num / den if den > 0 else 0
    num, den = sub_df[["EAL_VALP", "POPULATION"]].dropna().sum(axis=0).values
    ALR_VALP = num / den if den > 0 else 0
    num, den = sub_df[["EAL_VALA", "AGRIVALUE"]].dropna().sum(axis=0).values
    ALR_VALA = num / den if den > 0 else 0
    data.append([zid, ALR_VALB, ALR_VALP, ALR_VALA])

nri_data_zip = pd.DataFrame(data, columns=["ZIP", "ALR_VALB", "ALR_VALP", "ALR_VALA"])

In [10]:
nri_data_county.to_csv("derived_data/county/fema_eal.csv", index=False)
nri_data_zip.to_csv("derived_data/zcta/fema_eal.csv", index=False)
nri_data_tract.to_csv("derived_data/tract/fema_eal.csv", index=False)